# Layoff Cleaning and Analyzing 

In [6]:
import pandas as pd
df = pd.read_csv('layoffs (1).csv')


In [7]:
# Remove duplicate records
df["row_num"] = df.groupby(["company", "location", "industry", "total_laid_off", "date", "stage", "country", "funds_raised_millions"]).cumcount() + 1
df = df[df["row_num"] == 1]
df.drop(columns=["row_num"], inplace=True)

In [10]:
# Standardizing company names
df["company"] = df["company"].str.strip()

In [11]:
# Standardizing country names
df.loc[df["country"].str.startswith("United States", na=False), "country"] = "United States"


In [12]:
# Convert date column to datetime format
df["date"] = pd.to_datetime(df["date"], format='%m/%d/%Y', errors='coerce')

In [13]:
# Handling missing values in industry column
df.loc[df["industry"].str.strip() == "", "industry"] = None
industry_map = df.groupby("company")["industry"].first()
df["industry"] = df["company"].map(industry_map)

In [14]:
#  Removing rows where total_laid_off and percentage_laid_off are NULL
df = df.dropna(subset=["total_laid_off", "percentage_laid_off"], how="all")

In [17]:
# Handling Outliers in total_laid_off
Q1 = df["total_laid_off"].quantile(0.25)
Q3 = df["total_laid_off"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df = df[(df["total_laid_off"] >= lower_bound) & (df["total_laid_off"] <= upper_bound)]


In [19]:
df.head()


,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
3,UpGrad,Mumbai,Education,120.0,NaN,2023-03-06,Unknown,India,631.0
4,Loft,Sao Paulo,Real Estate,340.0,0.15,2023-03-03,Unknown,Brazil,788.0
5,Embark Trucks,SF Bay Area,Transportation,230.0,0.70,2023-03-03,Post-IPO,United States,317.0
6,Lendi,Sydney,Real Estate,100.0,NaN,2023-03-03,Unknown,Australia,59.0
7,UserTesting,SF Bay Area,Marketing,63.0,NaN,2023-03-03,Acquired,United States,152.0


In [20]:
# Step 9: Convert funds_raised_millions to numeric
df["funds_raised_millions"] = pd.to_numeric(df["funds_raised_millions"], errors="coerce")

/var/folders/b_/81pnbcn12j9_jgbwb9mnrym00000gn/T/ipykernel_19983/1658637383.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["funds_raised_millions"] = pd.to_numeric(df["funds_raised_millions"], errors="coerce")


In [23]:
# Checking dataset changes 
print("Data after cleaning :")
print(df.info())
print(df.head())

Data after cleaning :
<class 'pandas.core.frame.DataFrame'>
Index: 1326 entries, 3 to 2355
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   company                1326 non-null   object        
 1   location               1326 non-null   object        
 2   industry               1326 non-null   object        
 3   total_laid_off         1326 non-null   float64       
 4   percentage_laid_off    956 non-null    float64       
 5   date                   1326 non-null   datetime64[ns]
 6   stage                  1326 non-null   object        
 7   country                1326 non-null   object        
 8   funds_raised_millions  1326 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 103.6+ KB
None
         company     location        industry  total_laid_off  \
3         UpGrad       Mumbai       Education           120.0   
4           Loft    

In [24]:
max_layoffs = df["total_laid_off"].max()
print("Max layoffs in a single event:", max_layoffs)

Max layoffs in a single event: 370.0


In [25]:
max_percentage = df["percentage_laid_off"].max()
print("Max percentage of workforce laid off:", max_percentage)

Max percentage of workforce laid off: 1.0


In [28]:
high_impact_layoffs = df[df["percentage_laid_off"] == 1].sort_values("funds_raised_millions", ascending=False)
print("Companies with 100% layoffs ordered by funds raised:")
print(high_impact_layoffs.head())

Companies with 100% layoffs ordered by funds raised:
                  company     location        industry  total_laid_off  \
340           Britishvolt       London  Transportation           206.0   
699   Deliveroo Australia    Melbourne            Food           120.0   
145               Openpay    Melbourne         Finance            83.0   
1001         Simple Feast   Copenhagen            Food           150.0   
1051                Reali  SF Bay Area     Real Estate           140.0   

      percentage_laid_off       date     stage         country  \
340                   1.0 2023-01-17   Unknown  United Kingdom   
699                   1.0 2022-11-15  Post-IPO       Australia   
145                   1.0 2023-02-07  Post-IPO       Australia   
1001                  1.0 2022-09-07   Unknown         Denmark   
1051                  1.0 2022-08-24  Series B   United States   

      funds_raised_millions  
340                  2400.0  
699                  1700.0  
145            

In [31]:
company_layoffs = df.groupby("company")["total_laid_off"].sum().reset_index().sort_values("total_laid_off", ascending=False)
print("Total layoffs per company:")
print(company_layoffs.head())

Total layoffs per company:
        company  total_laid_off
1063     WeWork          1150.0
558        Loft           905.0
1052      Vroom           882.0
105       Blend           860.0
263   Deliveroo           717.0


In [32]:
date_range = df["date"].min(), df["date"].max()
print("Date range of layoffs:", date_range)

Date range of layoffs: (Timestamp('2020-03-11 00:00:00'), Timestamp('2023-03-06 00:00:00'))


In [33]:
industry_layoffs = df.groupby("industry")["total_laid_off"].max().reset_index().sort_values("total_laid_off", ascending=False)
print("Industry with highest layoffs:")
print(industry_layoffs.head())

Industry with highest layoffs:
          industry  total_laid_off
20       Marketing           370.0
12            Food           367.0
10         Finance           365.0
30  Transportation           360.0
26          Retail           360.0


In [34]:
country_layoffs = df.groupby("country")["total_laid_off"].sum().reset_index().sort_values("total_laid_off", ascending=False)
print("Total layoffs by country:")
print(country_layoffs.head())

Total layoffs by country:
          country  total_laid_off
39  United States         75756.0
14          India         13574.0
3          Brazil          6225.0
12        Germany          4412.0
4          Canada          4366.0


In [35]:
yearly_layoffs = df.groupby(df["date"].dt.year)["total_laid_off"].sum().reset_index().sort_values("date", ascending=False)
print("Yearly layoffs:")
print(yearly_layoffs)


Yearly layoffs:
   date  total_laid_off
3  2023         26348.0
2  2022         64065.0
1  2021          2399.0
0  2020         30650.0


In [36]:
monthly_layoffs = df.groupby(df["date"].dt.to_period("M"))["total_laid_off"].sum().reset_index().sort_values("date")
monthly_layoffs["rolling_total"] = monthly_layoffs["total_laid_off"].cumsum()
print("Monthly layoffs with rolling total:")
print(monthly_layoffs)

Monthly layoffs with rolling total:
       date  total_laid_off  rolling_total
0   2020-03          6165.0         6165.0
1   2020-04         12362.0        18527.0
2   2020-05          5324.0        23851.0
3   2020-06          2601.0        26452.0
4   2020-07          1777.0        28229.0
5   2020-08           969.0        29198.0
6   2020-09           609.0        29807.0
7   2020-10           450.0        30257.0
8   2020-11           219.0        30476.0
9   2020-12           174.0        30650.0
10  2021-01           536.0        31186.0
11  2021-02           855.0        32041.0
12  2021-03            47.0        32088.0
13  2021-04           261.0        32349.0
14  2021-07            80.0        32429.0
15  2021-08            67.0        32496.0
16  2021-09           161.0        32657.0
17  2021-10            22.0        32679.0
18  2021-11            70.0        32749.0
19  2021-12           300.0        33049.0
20  2022-01           510.0        33559.0
21  2022-02       

In [37]:
company_year_layoffs = df.groupby(["company", df["date"].dt.year])["total_laid_off"].sum().reset_index().sort_values("total_laid_off", ascending=False)
print("Company layoffs per year:")
print(company_year_layoffs.head())

Company layoffs per year:
         company  date  total_laid_off
1168      WeWork  2020           850.0
635   MakeMyTrip  2020           700.0
1155       Vroom  2022           607.0
57       Argo AI  2022           582.0
296         Doma  2022           560.0


In [38]:
company_year_ranked = company_year_layoffs.copy()
company_year_ranked["rank"] = company_year_ranked.groupby("date")["total_laid_off"].rank(method="dense", ascending=False)
top_companies_per_year = company_year_ranked[company_year_ranked["rank"] <= 5]
print("Top 5 companies per year with highest layoffs:")
print(top_companies_per_year)

Top 5 companies per year with highest layoffs:
            company  date  total_laid_off  rank
1168         WeWork  2020           850.0   1.0
635      MakeMyTrip  2020           700.0   2.0
1155          Vroom  2022           607.0   1.0
57          Argo AI  2022           582.0   2.0
296            Doma  2022           560.0   3.0
1102          Udaan  2022           530.0   4.0
108           Blend  2022           520.0   5.0
1185            Wix  2023           370.0   1.0
281       Deliveroo  2020           367.0   3.0
1121        Upstart  2023           365.0   2.0
923      Shutterfly  2023           360.0   3.0
670         Moladin  2023           360.0   3.0
425            Grab  2020           360.0   4.0
1232          iFood  2023           355.0   4.0
1005         Swiggy  2020           350.0   5.0
282       Deliveroo  2023           350.0   5.0
305         Dropbox  2021           315.0   1.0
284   Delivery Hero  2021           300.0   2.0
1050        ThredUp  2021           243.0

In [39]:
# Save the cleaned dataset
df.to_csv("layoffs_cleaned.csv", index=False)